In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

import qgrid

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()


In [2]:
# Parameters
years_in = [1900, 2022]

In [3]:
def read_data(fname, start_year, end_year):
    df = pd.read_csv(fname).set_index('country')
    df = df.replace({'[bB]': '*1e9', '[kK]': '*1e3', '[Mm]': '*1e6'}, regex=True)
    df = df.T.loc[str(start_year):str(end_year)].apply(lambda x: x.map(pd.eval).astype(int))
    return df

In [4]:
# Another approximation
interesting_values = ['population', 'electricity_generation', 'renewables_electricity', 'fossil_electricity']

# Load file
df_energy = pd.read_csv('./data/energy_data.csv')
# Get interesting columns
df_energy = df_energy[['country', 'iso_code', 'year'] + interesting_values]
# Remove rows with continents, unexisting countries.
df_energy = df_energy[pd.notna(df_energy['iso_code'])]
# Create separated tables pivotting
separated_df = {value: df_energy.pivot(columns='country', index='year', values=value) for value in interesting_values} 
df_energy[df_energy['country'] == 'World']

,country,iso_code,year,population,electricity_generation,renewables_electricity,fossil_electricity
16904,World,OWID_WRL,1900,1.650000e+09,NaN,NaN,NaN
16905,World,OWID_WRL,1901,1.659948e+09,NaN,NaN,NaN
16906,World,OWID_WRL,1902,1.669997e+09,NaN,NaN,NaN
16907,World,OWID_WRL,1903,1.680115e+09,NaN,NaN,NaN
16908,World,OWID_WRL,1904,1.690269e+09,NaN,NaN,NaN
...,...,...,...,...,...,...,...
17020,World,OWID_WRL,2016,7.464022e+09,23925.445,5900.412,15496.799
17021,World,OWID_WRL,2017,7.547859e+09,24570.376,6276.014,15750.371
17022,World,OWID_WRL,2018,7.631091e+09,25559.572,6712.063,16233.117
17023,World,OWID_WRL,2019,7.713468e+09,25899.806,7065.089,16114.042


In [80]:
import geopandas


world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres')).to_crs("EPSG:4326")

ren_ener = df_energy[(df_energy['country'] != 'World') & (df_energy['year'] == 2010)]
ren_ener.rename(columns={'iso_code': 'iso_a3'}, inplace=True)

merged = world.merge(ren_ener, on='iso_a3').set_index('iso_a3')

fig = px.choropleth_mapbox(merged,
                           geojson=merged.geometry,
                           locations=merged.index,
                           mapbox_style="open-street-map",
                           zoom=1,
                           color="renewables_electricity")
fig.show()
# ren_ener

In [56]:
ren_ener = df_energy[(df_energy['country'] != 'World') & (df_energy['year'] >= 1990) & (df_energy['year'] < 2020)]
ren_ener['renew_per_capita'] = ren_ener['renewables_electricity'] / ren_ener['electricity_generation']
ren_ener['fos_per'] = ren_ener['fossil_electricity'] / ren_ener['electricity_generation']

fig = px.choropleth(ren_ener, locations="iso_code",
                    color="renew_per_capita", # lifeExp is a column of gapminder
                    hover_name="country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma, 
                    animation_frame="year",)
fig.show()

In [81]:
# GDP from other file
df_gdp = pd.read_csv('./data/gdp.csv', header=2)
df_gdp = df_gdp.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 66']).set_index('Country Name').T
df_gdp.index = df_gdp.index.astype(int)
df_gdp.iplot()
